In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_auto_diffy")' SchwiftyNotebook_auto_diffy

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_auto_diffy")
		SchwiftyNotebook_auto_diffy
With SwiftPM flags: []
Working in: /tmp/tmpo56r5bd5/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
Updating https://github.com/mxcl/Path.swift
Updating https://github.com/saeta/Just
Updating https://github.com/latenitesoft/NotebookExport
Completed resolution in 0.95s
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availab

In [9]:
//export
import TensorFlow
import SchwiftyNotebook_auto_diffy

In [6]:
import Python
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

public let plotter = Python.import("matplotlib.pyplot")

# Data Setup

In [ ]:
// https://www.kaggle.com/austinreese/craigslist-carstrucks-data/downloads/craigslist-carstrucks-data.zip/8

func loadCarData(training: Bool, path: Path) -> TensorFloat {
    let batch = training ? 60000 : 10000
    
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let file = path/fname
    if !file.exists {
        downloadFile("https://www.kaggle.com/austinreese/craigslist-carstrucks-data/downloads/craigslist-carstrucks-data.zip/8",
                     dest:(path/"\(fname).gz").string)
        "/bin/unzip".shell((path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string))
    if labels { return data.asTensor() }
    else      { return data.asTensor().reshaped(to: shape)}
}
